In [1]:
import numpy as np
from functools import partial
import matplotlib.pyplot as plt 
import sympy as sp
from sympy.utilities.lambdify import lambdify

In [ ]:
!conda install pytorch torchvision cudatoolkit=9.0 -c pytorch

In [2]:
Y=np.load('tomatoes.npy') # Make sure you downloaded this file from Learn!
print('Length of data: ' + str(len(Y)))
print('Data:')
print(Y)

Length of data: 30
Data:
[13.27597167 15.08565281 13.41061844 11.5510474  10.63232069 13.76932811
 10.31644803 14.55388057 12.57937117 14.13647951 15.37354413  9.96525246
 12.66541964 11.33164599 12.14113179 11.57753389 15.86128067 10.23999543
 16.36971379 13.96021609 17.27321719 12.23476479 17.96792692 13.92888826
 10.64602864 16.45922776 15.56009798 13.02118014 16.50556431 14.08484862]


In [ ]:
def run_simulation_y( q0,p0, Nsteps, h, gamma, step_function, force_function, y):
    
    q_traj = [np.copy(q0)] 
    p_traj = [np.copy(p0)]
    t_traj = [0]

    q = np.copy(q0) 
    p = np.copy(p0)
    t = 0 

    for n in range(Nsteps):
        q,p = step_function(q, p, h, gamma, force_function,y)
        t = t + h 

        q_traj += [q] 
        p_traj += [p]   
        t_traj += [t] 

    q_traj = np.array(q_traj) 
    p_traj = np.array(p_traj) 
    t_traj = np.array(t_traj) 

    return q_traj, p_traj, t_traj

In [17]:
def prior(q):
    m1,m2,m3 = q 
    return sp.exp(-((13.5-m1)**2)/32)*sp.exp(-((13.5-m2)**2)/32)*sp.exp(-((13.5-m3)**2)/32)
def likelihood(q,y):
    m1,m2,m3 = q
    return sp.exp(-((y-m1)**2)/2)+sp.exp(-((y-m2)**2)/2)+sp.exp(-((y-m3)**2)/2)
def U(q,y):
    func = lambdify((m1,m2,m3,y), likelihood(q,y) ,'numpy') # returns a numpy-ready function
    mu1,mu2,mu3 = q
    arr = func(mu1,mu2,mu3,y)
    pri = lambdify((m1,m2,m3),prior(q),'numpy')
    pri = pri(q[0],q[1],q[2])
    V = -np.log(pri)#-sum(arr)
    return V
def force(q,y):
    muA,muB,muB = q
    m1,m2,m3 = sp.symbols('m1 m2 m3',real=True)

    dLike1 = sp.diff(likelihood([m1,m2,m3],y),m1)
    dPrior1= sp.diff(-sp.log(prior(q)),m1)

    func = lambdify((m1,m2,m3,y), dLike1 ,'numpy') # returns a numpy-ready function
    arr = func(muA,muB,muC,y)
    
    dU_dm1 = dPrior1.subs(m1,muA).subs(m2,muB).subs(m3,muC) - sum(arr)
    
    return dU_dm1.subs(m1,muA).subs(m2,muB).subs(m3.mu3).subs(y,y)




In [18]:
# m1,m2,m3 = sp.symbols('m1 m2 m3',real=True)
# dLike1 = sp.diff(likelihood([m1,m2,m3],Y),m1)
# print(like(12,13,14,Y))
# U( [12,13,14], Y) - U( [11,12,13], Y)
force([12,13,14],Y)
# func = lambdify((m1,m2,m3,y), likelihood(q,y) ,'numpy') # returns a numpy-ready function
# a = func(12,13,14,Y)
print(dLike1)
# a = [12,13,14]
# m1,m2,m3 = a
# print(m1)

SyntaxError: invalid syntax (<string>, line 1)

In [ ]:
q = [m1,m2,m3]
m1,m2,m3 = sp.symbols('m1 m2 m3',real=True)
dU_dm1 =sp.diff(prior(q)*likelihood(q,Y),m1)
# Values of 
mu1 = 11; mu2 = 13; mu3 = 15

In [ ]:
def gradient(q,y):
    m1,m2,m3 = q
    
    m1,m2,m3 = sp.symbols('m1 m2 m3',real=True)
    dU_dm1 =sp.diff(prior(q)*sum(likelihood(q,Y)),m1)
    
    # constant number in the likelihood part of all derivatives
    denom = np.exp(-((y-m1)**2)/2)+np.exp(-((y-m2)**2)/2)+np.exp(-((y-m3)**2)/2)
    
    # should it be F = -dU_dq
    dU_dm1 = (13.5-m1)/16 + sum((y-m1)*np.exp(-((y-m1)**2)/2)/denom)
    dU_dm2 = (13.5-m2)/16 + sum((y-m2)*np.exp(-((y-m2)**2)/2)/denom)
    dU_dm3 = (13.5-m3)/16 + sum((y-m3)*np.exp(-((y-m3)**2)/2)/denom)
    
    dU_dq = np.array([dU_dm1,dU_dm2,dU_dm3])
    return dU_dq